In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import tqdm
from Bio import SeqIO

from transformers import PreTrainedTokenizerFast

from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.processors import TemplateProcessing

import sentencepiece as spm


DATASET = '240212_plasmid_seq_54646.fasta'
DATASET_TXT = '240212_plasmid_seq_54646.txt'
DATASET_DUMMY = '240212_plasmid_seq_54646_dummy.txt'
TOKENIZER = 'dna_bpe_tokenizer'

VOCAB_SIZE = 4096
NUM_SEQUENCES = 10     #54646
MAX_TOKEN_LENGTH = 32
SPECIAL_TOKENS = ['[UNK]', '[SEP]', '[PAD]', '[CLS]', '[MASK]']

In [ ]:
def read_fasta_to_dataframe(file_path):
    sequences = []
    for record in SeqIO.parse(file_path, "fasta"):
        sequences.append([record.id, str(record.seq), record.description])
    
    dataset = pd.DataFrame(sequences, columns=['ID', 'Sequence', 'Description'])
    return dataset


# Read plasmid dataset
dataset = read_fasta_to_dataframe(DATASET)

# Remove new line and space characters from DNA sequences
dataset['Sequence'] = dataset['Sequence'].transform(
    lambda seq: seq.replace(' ', '').replace('\n', '').replace('\r', '').replace('\t', '')
)

# Save dataset to txt for tokenizer
# dataset['Sequence'].to_csv(DATASET_TXT, index=False, header=False)
dataset['Sequence'].iloc[:NUM_SEQUENCES].to_csv(DATASET_DUMMY, index=False, header=False)

In [ ]:
dataset

In [ ]:
# Train the SentencePiece model with HuggingFace

tokenizer = Tokenizer(
    BPE(unk_token="[UNK]")
)

# Set pre-tokenizer
tokenizer.pre_tokenizer = Whitespace()

# Train tokenizer
trainer = BpeTrainer(
    vocab_size=VOCAB_SIZE,
    min_frequency=2,
    show_progress=True,
    special_tokens=SPECIAL_TOKENS,
    initial_alphabet=["A", "T", "C", "G"],
    max_token_length=MAX_TOKEN_LENGTH
)

tokenizer.train([DATASET_DUMMY], trainer)

# Set post-processor with correct special token references
tokenizer.post_processor = TemplateProcessing(
    single="[CLS] $A [SEP]",
    pair="[CLS] $A [SEP] $B:1 [SEP]:1",
    special_tokens=[
        ("[CLS]", tokenizer.token_to_id("[CLS]")),
        ("[SEP]", tokenizer.token_to_id("[SEP]")),
        ("[PAD]", tokenizer.token_to_id("[PAD]")),
        ("[UNK]", tokenizer.token_to_id("[UNK]")),
        ("[MASK]", tokenizer.token_to_id("[MASK]")),
    ]
)

tokenizer.save(f"{TOKENIZER}.json")

In [ ]:
# Load the SentencePiece tokenizer

tokenizer = PreTrainedTokenizerFast(
    tokenizer_file=f"{TOKENIZER}.json"
)

In [42]:
# Test the tokenizer on a sample DNA sequence
sequence = "ATTCTGCGGTTCCCCCTGGAAGACCTACGCAAGTTGGGCCAGCTCAGAGGTGGAATCAACGAAGGCGAGC"
encoded = tokenizer(sequence)
print("Encoded sequence:", encoded)

# Decode the tokens back to the original sequence
decoded_sequence = tokenizer.decode(encoded['input_ids'])
print("Decoded sequence:", decoded_sequence)

Encoded sequence: {'input_ids': [3, 5, 532, 711, 40, 133, 86, 2202, 530, 41, 229, 132, 133, 269, 27, 1094, 1], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
Decoded sequence: [CLS] A TTCTGC GGTTCC CCC TGGAA GACC TACGCAA GTTGG GCCA GCTCA GAGG TGGAA TCAAC GAA GGCGAGC [SEP]


In [ ]:
sorted(tokenizer.vocab.keys())

In [ ]:
#################### DEAD ZONE - DO NOT ENTER ####################
# dataset['Sequence'].transform(len).plot(kind='hist', bins=int(1e3))
# plt.xlim(0, 1e6)
# plt.show()

# # Train the SentencePiece model with spm

# with tqdm(total=NUM_SEQUENCES, desc="Training Tokenizer...", unit="sequences") as pbar:

#     spm.SentencePieceTrainer.train(
#         input=DATASET_TXT,                      # Input file containing DNA sequences
#         model_prefix=TOKENIZER,                 # Prefix for the output model files
#         vocab_size=VOCAB_SIZE,                  # Vocabulary size
#         model_type='bpe',                       # Model type (BPE)
#         pad_id=0,                               # ID for padding token
#         unk_id=1,                               # ID for unknown token
#         bos_id=2,                               # ID for beginning-of-sequence token
#         eos_id=3,                               # ID for end-of-sequence token
#         user_defined_symbols=SPECIAL_TOKENS,    # Special tokens
#         character_coverage=1.0,                 # Ensure full coverage of the input characters
#         input_sentence_size=NUM_SEQUENCES,      # Limit the number of sentences for training for efficiency
#         shuffle_input_sentence=True             # Shuffle the input sentences to improve training
#     )
#     pbar.update(NUM_SEQUENCES)

# # Load the SentencePiece model
# sp = spm.SentencePieceProcessor()
# sp.load(f'{TOKENIZER}.model')

# # Tokenize a sentence
# sequence = "ATTCTGCGGTTCCCCCTGGAAGACCTACGCAAGTTGGGCCAGCTCAGAGGTGGAATCAACGAAGGCGAGC"
# tokens = sp.encode_as_pieces(sequence)
# print("Tokens:", tokens)

# # Convert tokens back to text
# decoded_text = sp.decode_pieces(tokens)
# print("Decoded text:", decoded_text)